In [1]:
import pandas as pd
import numpy as np
from itertools import product

class QuizGridSearchAnalyzer:
    def __init__(self):
        # 기본 비용/수익 설정 (변수로 변경될 수 있음)
        self.costs = {
            'ad_revenue': 1,           # 광고 수익 (변수)
            'success_reward': 3,       # 일반 보상 (고정)
            'retry_success_reward': 1, # 재도전 성공 보상 (고정)
            'streak_reward': 100,      # 7일 연속 참여 보상 (변수)
            'extra_reward': 3          # 추가 보상 (변수)
        }
        
        # 7일 연속 참여 분포 (17.5% 트리거 확률)
        self.streak_7_trigger_rate = 0.175
        self.streak_7_distribution = {
            1: 0.4911, 2: 0.1823, 3: 0.0976, 4: 0.0589, 5: 0.0421,
            6: 0.0283, 7: 0.0217, 8: 0.0148, 9: 0.0154, 10: 0.0123,
            11: 0.0161, 12: 0.0193
        }
        
        self.daily_users = 100000
    
    def calculate_success_scenario(self, p_correct, p_watch_ad, p_extra_reward, p_retry, ad_revenue, extra_reward):
        """정답 즉시 성공 시나리오 계산"""
        # 광고 시청 수 (정답자 중 광고 시청하는 비율)
        ad_views = p_watch_ad
        
        # 추가 보상 획득 수 (광고 시청자 중 추가 보상 받는 비율)
        extra_rewards = p_watch_ad * p_extra_reward
        
        # 수익/비용
        revenue = ad_views * ad_revenue
        cost = self.costs['success_reward'] + extra_rewards * extra_reward
        
        return {
            'ad_views': ad_views,
            'extra_rewards': extra_rewards,
            'revenue': revenue,
            'cost': cost,
            'profit': revenue - cost
        }
    
    def calculate_failure_scenario(self, p_correct, p_watch_ad, p_extra_reward, p_retry, ad_revenue, extra_reward):
        """오답 후 재도전 시나리오 계산"""
        total_ad_views = 1  # 첫 강제 광고
        total_extra_rewards = 0
        total_retry_success = 0
        
        retry_chance = p_retry
        max_retries = 10
        
        for attempt in range(1, max_retries + 1):
            # 이번 시도에서 성공할 확률
            success_prob = retry_chance * p_correct
            
            if success_prob > 0.001:  # 무시할 수 있는 수준까지
                # 재도전 성공 시
                total_retry_success += success_prob
                
                # 성공 후 광고 시청 및 추가 보상
                ad_after_success = success_prob * p_watch_ad
                extra_after_success = ad_after_success * p_extra_reward
                
                total_ad_views += ad_after_success
                total_extra_rewards += extra_after_success
            
            # 실패하고 재도전할 확률
            fail_and_retry_prob = retry_chance * (1 - p_correct) * p_retry
            
            if fail_and_retry_prob < 0.001:  # 무시할 수 있는 수준
                break
            
            # 실패 시 추가 광고
            total_ad_views += fail_and_retry_prob
            
            # 다음 재도전 확률 업데이트
            retry_chance = fail_and_retry_prob
        
        # 수익/비용 계산
        revenue = total_ad_views * ad_revenue
        cost = total_retry_success * self.costs['retry_success_reward'] + \
               total_extra_rewards * extra_reward
        
        return {
            'ad_views': total_ad_views,
            'retry_success': total_retry_success,
            'extra_rewards': total_extra_rewards,
            'revenue': revenue,
            'cost': cost,
            'profit': revenue - cost
        }
    
    def calculate_7day_streak_metrics(self, streak_reward):
        """7일 연속 참여 보상 계산"""
        adjusted_users = self.daily_users * self.streak_7_trigger_rate
        
        total_streak_users = 0
        total_streak_cost = 0
        
        for streak_count, probability in self.streak_7_distribution.items():
            expected_users = adjusted_users * probability
            streak_cost = expected_users * streak_count * streak_reward
            
            total_streak_users += expected_users
            total_streak_cost += streak_cost
        
        return {
            'daily_streak_users': total_streak_users,
            'daily_streak_cost': total_streak_cost
        }
    
    def calculate_scenario_metrics(self, p_correct, p_watch_ad, p_extra_reward, p_retry, ad_revenue, streak_reward, extra_reward):
        """단일 시나리오의 모든 지표 계산"""
        
        # 성공/실패 시나리오별 계산
        success_result = self.calculate_success_scenario(p_correct, p_watch_ad, p_extra_reward, p_retry, ad_revenue, extra_reward)
        failure_result = self.calculate_failure_scenario(p_correct, p_watch_ad, p_extra_reward, p_retry, ad_revenue, extra_reward)
        
        # 일일 사용자 분포
        daily_success_users = self.daily_users * p_correct
        daily_failure_users = self.daily_users * (1 - p_correct)
        
        # 일일 집계 지표
        일일_광고시청수 = int(daily_success_users * success_result['ad_views'] + 
                         daily_failure_users * failure_result['ad_views'])
        
        일일_참여수 = self.daily_users
        
        일일_일반보상지급수 = int(daily_success_users)
        
        일일_재도전성공수 = int(daily_failure_users * failure_result['retry_success'])
        
        일일_추가보상수 = int(daily_success_users * success_result['extra_rewards'] +
                         daily_failure_users * failure_result['extra_rewards'])
        
        # 7일 연속 참여 보상
        streak_metrics = self.calculate_7day_streak_metrics(streak_reward)
        일일_7일연속보상수 = int(streak_metrics['daily_streak_users'])
        
        # 보상액 계산
        일반보상액 = 일일_일반보상지급수 * self.costs['success_reward']
        재도전보상액 = 일일_재도전성공수 * self.costs['retry_success_reward']
        추가보상액 = 일일_추가보상수 * extra_reward
        연속보상액 = streak_metrics['daily_streak_cost']
        총보상액 = 일반보상액 + 재도전보상액 + 추가보상액 + 연속보상액
        
        # 매출액
        광고매출액 = 일일_광고시청수 * ad_revenue
        총매출액 = 광고매출액
        
        # 손익 계산
        일일순손익액 = 총매출액 - 총보상액
        주간게임손익액 = (광고매출액 - (일반보상액 + 재도전보상액 + 추가보상액)) * 7
        주간순손익액 = 주간게임손익액 - 연속보상액
        수익성여부 = '수익' if 주간순손익액 > 0 else '손실'
        
        return {
            '광고매출': ad_revenue,
            '7일연속보상': streak_reward,
            '추가보상': extra_reward,
            '정답률': f"{p_correct:.1%}",
            '광고시청률': f"{p_watch_ad:.1%}",
            '추가보상률': f"{p_extra_reward:.1%}",
            '재도전률': f"{p_retry:.1%}",
            '일일_광고시청수': 일일_광고시청수,
            '일일_참여수': 일일_참여수,
            '일일_일반보상지급수': 일일_일반보상지급수,
            '일일_재도전성공수': 일일_재도전성공수,
            '일일_추가보상수': 일일_추가보상수,
            '일일_7일연속보상수': 일일_7일연속보상수,
            '일반보상액': 일반보상액,
            '재도전보상액': 재도전보상액,
            '추가보상액': 추가보상액,
            '7일연속보상액': int(연속보상액),
            '총보상액': int(총보상액),
            '광고매출액': 광고매출액,
            '총매출액': 총매출액,
            '일일순손익액': int(일일순손익액),
            '주간순손익액': int(주간순손익액),
            '수익성여부': 수익성여부
        }
    
    def run_grid_search(self):
        """전체 Grid Search 실행 (비용/매출 변수 포함)"""
        
        # Grid 범위 정의
        correct_rates = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
        watch_ad_rates = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
        extra_reward_rates = [0.5, 0.6, 0.7, 0.8, 0.9]
        retry_rates = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
        
        # 비용/매출 변수 범위 추가
        ad_revenues = [1, 2, 3, 4, 5]  # 광고 매출 1~5원
        streak_rewards = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # 7일 연속 보상 10~100원
        extra_rewards = [1, 2, 3]  # 추가 보상 1~3원
        
        total_scenarios = (len(correct_rates) * len(watch_ad_rates) * len(extra_reward_rates) * 
                          len(retry_rates) * len(ad_revenues) * len(streak_rewards) * len(extra_rewards))
        
        print(f"Grid Search 시작:")
        print(f"확률 변수: {len(correct_rates)}×{len(watch_ad_rates)}×{len(extra_reward_rates)}×{len(retry_rates)}")
        print(f"비용/매출 변수: {len(ad_revenues)}×{len(streak_rewards)}×{len(extra_rewards)}")
        print(f"총 시나리오: {total_scenarios:,}개")
        
        results = []
        scenario_id = 1
        
        # 모든 조합에 대해 계산
        for ad_revenue, streak_reward, extra_reward in product(ad_revenues, streak_rewards, extra_rewards):
            for p_correct, p_watch_ad, p_extra_reward, p_retry in product(
                correct_rates, watch_ad_rates, extra_reward_rates, retry_rates
            ):
                metrics = self.calculate_scenario_metrics(
                    p_correct, p_watch_ad, p_extra_reward, p_retry,
                    ad_revenue, streak_reward, extra_reward
                )
                
                # 시나리오 ID 추가
                metrics['시나리오_ID'] = f"S{scenario_id:06d}"
                
                results.append(metrics)
                scenario_id += 1
                
                # 진행상황 출력
                if scenario_id % 10000 == 0:
                    progress = (scenario_id-1) / total_scenarios * 100
                    print(f"진행상황: {scenario_id-1:,}개 완료 ({progress:.1f}%)...")
        
        # DataFrame 생성 (컬럼 순서 조정)
        column_order = [
            '시나리오_ID', '광고매출', '7일연속보상', '추가보상',
            '정답률', '광고시청률', '추가보상률', '재도전률',
            '일일_광고시청수', '일일_참여수', '일일_일반보상지급수', '일일_재도전성공수', 
            '일일_추가보상수', '일일_7일연속보상수',
            '일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액',
            '광고매출액', '총매출액', '일일순손익액', '주간순손익액', '수익성여부'
        ]
        
        df = pd.DataFrame(results)[column_order]
        
        print(f"Grid Search 완료: {len(df):,}개 시나리오 생성")
        print(f"수익 시나리오: {len(df[df['수익성여부'] == '수익']):,}개")
        print(f"손실 시나리오: {len(df[df['수익성여부'] == '손실']):,}개")
        
        return df

def extract_best_scenarios(df, base_scenario_params=None, n=10, sort_by='주간순손익액', improvement_threshold=0):
    """
    기존 대비 가장 좋은 시나리오를 n개 추출하는 함수
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Grid Search 결과 데이터프레임
    base_scenario_params : dict, optional
        기준 시나리오 파라미터. None이면 기본값 사용
    n : int
        추출할 시나리오 개수 (기본값: 10)
    sort_by : str
        정렬 기준 컬럼 (기본값: '주간순손익액')
    improvement_threshold : int
        최소 개선 금액 기준 (기본값: 0원)
    
    Returns:
    --------
    dict : 분석 결과를 포함한 딕셔너리
    """
    
    # 필수 컬럼 확인
    required_columns = ['시나리오_ID', sort_by]
    if not all(col in df.columns for col in required_columns):
        print(f"❌ 필수 컬럼이 없습니다: {[col for col in required_columns if col not in df.columns]}")
        return None
    
    # 기본 시나리오 파라미터 설정
    if base_scenario_params is None:
        base_scenario_params = {
            '광고매출': 1,
            '7일연속보상': 100,
            '추가보상': 3,
            '정답률': '50.0%',
            '광고시청률': '50.0%',
            '추가보상률': '70.0%',
            '재도전률': '50.0%'
        }
    
    # 기준 시나리오 찾기 (존재하는 컬럼만 사용)
    base_scenario = df.copy()
    valid_params = {}
    
    for key, value in base_scenario_params.items():
        if key in df.columns:
            base_scenario = base_scenario[base_scenario[key] == value]
            valid_params[key] = value
        else:
            print(f"⚠️ 컬럼 '{key}'이 존재하지 않아 조건에서 제외됩니다.")
    
    if base_scenario.empty:
        print("❌ 기준 시나리오를 찾을 수 없습니다.")
        print(f"유효한 검색 조건: {valid_params}")
        return None
    
    base_profit = base_scenario.iloc[0][sort_by]
    base_scenario_id = base_scenario.iloc[0]['시나리오_ID']
    
    print(f"=== 기준 시나리오 ({base_scenario_id}) ===")
    
    # 존재하는 컬럼만 출력
    cost_vars = ['광고매출', '7일연속보상', '추가보상']
    prob_vars = ['정답률', '광고시청률', '추가보상률', '재도전률']
    
    cost_info = []
    for var in cost_vars:
        if var in df.columns:
            cost_info.append(f"{var}={base_scenario.iloc[0][var]}")
    
    prob_info = []
    for var in prob_vars:
        if var in df.columns:
            prob_info.append(base_scenario.iloc[0][var])
    
    if cost_info:
        print(f"설정: {', '.join(cost_info)}")
    if prob_info:
        print(f"확률: {', '.join(map(str, prob_info))}")
    
    print(f"기준 {sort_by}: {base_profit:,}원")
    
    if '수익성여부' in df.columns:
        print(f"수익성: {base_scenario.iloc[0]['수익성여부']}")
    
    # 개선된 시나리오 필터링
    improved_scenarios = df[df[sort_by] > base_profit + improvement_threshold].copy()
    
    if improved_scenarios.empty:
        print(f"\n❌ 개선 기준({improvement_threshold:,}원)을 만족하는 시나리오가 없습니다.")
        return {
            'base_scenario': base_scenario.iloc[0],
            'improved_scenarios': pd.DataFrame(),
            'analysis': {}
        }
    
    # 개선 금액 계산
    improved_scenarios['개선금액'] = improved_scenarios[sort_by] - base_profit
    if base_profit > 0:
        improved_scenarios['개선배수'] = improved_scenarios[sort_by] / base_profit
    else:
        improved_scenarios['개선배수'] = float('inf')
    
    # 상위 n개 추출
    best_scenarios = improved_scenarios.nlargest(n, sort_by).copy()
    
    print(f"\n=== 상위 {len(best_scenarios)}개 개선 시나리오 ===")
    print(f"총 {len(improved_scenarios):,}개 시나리오가 기준보다 우수함")
    
    # 결과 테이블 출력 (존재하는 컬럼만)
    display_columns = ['시나리오_ID']
    
    for col in cost_vars + prob_vars + [sort_by, '개선금액']:
        if col in best_scenarios.columns:
            display_columns.append(col)
    
    if '수익성여부' in best_scenarios.columns:
        display_columns.append('수익성여부')
    
    print(best_scenarios[display_columns].to_string(index=False))
    
    # 변수별 개선 영향 분석
    print(f"\n=== 변수별 개선 영향 분석 ===")
    
    variable_analysis = {}
    
    # 비용/매출 변수 분석 (존재하는 것만)
    for var in cost_vars:
        if var in df.columns:
            base_value = base_scenario.iloc[0][var]
            var_changes = best_scenarios[var] - base_value
            variable_analysis[var] = {
                '기준값': base_value,
                '평균변화': var_changes.mean(),
                '최대값': best_scenarios[var].max(),
                '최소값': best_scenarios[var].min(),
                '변화범위': f"{var_changes.min():+} ~ {var_changes.max():+}"
            }
            print(f"{var}: 기준 {base_value} → 범위 {best_scenarios[var].min()}~{best_scenarios[var].max()} (평균변화: {var_changes.mean():+.1f})")
    
    # 확률 변수 분석 (존재하는 것만)
    print(f"\n확률 변수 분포:")
    for var in prob_vars:
        if var in df.columns:
            var_dist = best_scenarios[var].value_counts().sort_index()
            if len(var_dist) > 0:
                most_common = var_dist.index[0]
                print(f"{var}: 최빈값 {most_common} ({var_dist.iloc[0]}회), 전체 {len(var_dist)}개 값")
    
    return {
        'base_scenario': base_scenario.iloc[0],
        'best_scenarios': best_scenarios,
        'improved_count': len(improved_scenarios),
        'variable_analysis': variable_analysis
    }

def analyze_scenario_comparison(df, scenario_ids):
    """
    특정 시나리오들을 상세 비교 분석하는 함수
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Grid Search 결과 데이터프레임
    scenario_ids : list
        비교할 시나리오 ID 리스트
    """
    scenarios = df[df['시나리오_ID'].isin(scenario_ids)].copy()
    
    if scenarios.empty:
        print("❌ 해당 시나리오들을 찾을 수 없습니다.")
        return
    
    print(f"=== 시나리오 상세 비교 ({len(scenarios)}개) ===")
    
    # 주요 지표 비교 (존재하는 컬럼만)
    comparison_columns = ['시나리오_ID']
    
    # 비용/매출 변수
    for col in ['광고매출', '7일연속보상', '추가보상']:
        if col in df.columns:
            comparison_columns.append(col)
    
    # 확률 변수
    for col in ['정답률', '광고시청률', '추가보상률', '재도전률']:
        if col in df.columns:
            comparison_columns.append(col)
    
    # 결과 지표
    for col in ['일일_광고시청수', '일일순손익액', '주간순손익액', '수익성여부']:
        if col in df.columns:
            comparison_columns.append(col)
    
    print(scenarios[comparison_columns].to_string(index=False))
    
    # 수익 구조 분석 (존재하는 컬럼만)
    print(f"\n=== 수익 구조 분석 ===")
    
    for idx, row in scenarios.iterrows():
        print(f"\n{row['시나리오_ID']}:")
        
        # 매출 정보
        if '광고매출액' in df.columns:
            print(f"  매출: 광고 {row['광고매출액']:,}원")
        
        # 비용 정보
        cost_info = []
        if '일반보상액' in df.columns:
            cost_info.append(f"일반 {row['일반보상액']:,}원")
        if '재도전보상액' in df.columns:
            cost_info.append(f"재도전 {row['재도전보상액']:,}원")
        if '추가보상액' in df.columns:
            cost_info.append(f"추가 {row['추가보상액']:,}원")
        if '7일연속보상액' in df.columns:
            cost_info.append(f"연속 {row['7일연속보상액']:,}원")
        
        if cost_info:
            total_cost = row['총보상액'] if '총보상액' in df.columns else 0
            print(f"  비용: {' + '.join(cost_info)} = {total_cost:,}원")
        
        if '주간순손익액' in df.columns:
            print(f"  순이익: {row['주간순손익액']:,}원")

def main():
    """메인 실행 함수"""
    
    # 분석기 생성 및 실행
    analyzer = QuizGridSearchAnalyzer()
    
    # 먼저 작은 샘플로 테스트
    print("=== 샘플 테스트 ===")
    test_metrics = analyzer.calculate_scenario_metrics(0.5, 0.5, 0.7, 0.5, 1, 100, 3)
    print(f"테스트 결과 키: {list(test_metrics.keys())}")
    print(f"광고매출 포함 여부: {'광고매출' in test_metrics}")
    print(f"7일연속보상 포함 여부: {'7일연속보상' in test_metrics}")
    print(f"추가보상 포함 여부: {'추가보상' in test_metrics}")
    
    # 전체 Grid Search 실행
    df = analyzer.run_grid_search()
    
    # 기본 통계 출력
    print("\n=== 기본 통계 ===")
    print(f"전체 시나리오 수: {len(df):,}")
    
    # 컬럼 존재 여부 확인
    print(f"DataFrame 컬럼: {list(df.columns)}")
    
    # 비용/매출 변수 확인
    cost_vars = ['광고매출', '7일연속보상', '추가보상']
    missing_cost_vars = [var for var in cost_vars if var not in df.columns]
    
    if missing_cost_vars:
        print(f"❌ 누락된 비용/매출 변수: {missing_cost_vars}")
    else:
        print(f"✅ 모든 비용/매출 변수 존재")
    
    if '수익성여부' in df.columns:
        print(f"수익 시나리오 수: {len(df[df['수익성여부'] == '수익']):,}")
        print(f"수익률: {len(df[df['수익성여부'] == '수익'])/len(df)*100:.1f}%")
    
    if '주간순손익액' in df.columns:
        print(f"\n주간 순손익 범위:")
        print(f"최대 수익: {df['주간순손익액'].max():,}원")
        print(f"최대 손실: {df['주간순손익액'].min():,}원")
        print(f"평균: {df['주간순손익액'].mean():,.0f}원")
    
    # 컬럼 확인 후 상위 시나리오 출력
    required_columns = ['시나리오_ID', '정답률', '광고시청률', '추가보상률', '재도전률', '주간순손익액']
    
    # 비용/매출 변수가 있으면 추가
    available_cost_vars = [var for var in cost_vars if var in df.columns]
    display_columns = ['시나리오_ID'] + available_cost_vars + ['정답률', '광고시청률', '추가보상률', '재도전률', '주간순손익액']
    
    # 실제 존재하는 컬럼만 필터링
    final_display_columns = [col for col in display_columns if col in df.columns]
    
    if '주간순손익액' in df.columns:
        # 상위 10개 수익 시나리오
        print(f"\n=== 상위 10개 수익 시나리오 ===")
        print(f"표시 컬럼: {final_display_columns}")
        top10 = df.nlargest(10, '주간순손익액')[final_display_columns]
        print(top10.to_string(index=False))
    
    # 기본 시나리오 찾기 (존재하는 컬럼만 사용)
    base_conditions = []
    
    if '광고매출' in df.columns:
        base_conditions.append(('광고매출', 1))
    if '7일연속보상' in df.columns:
        base_conditions.append(('7일연속보상', 100))
    if '추가보상' in df.columns:
        base_conditions.append(('추가보상', 3))
    if '정답률' in df.columns:
        base_conditions.append(('정답률', '50.0%'))
    if '광고시청률' in df.columns:
        base_conditions.append(('광고시청률', '50.0%'))
    if '추가보상률' in df.columns:
        base_conditions.append(('추가보상률', '70.0%'))
    if '재도전률' in df.columns:
        base_conditions.append(('재도전률', '50.0%'))
    
    if base_conditions:
        base_scenario = df.copy()
        for col, val in base_conditions:
            base_scenario = base_scenario[base_scenario[col] == val]
        
        if not base_scenario.empty:
            print(f"\n=== 기본 시나리오 ({base_scenario.iloc[0]['시나리오_ID']}) ===")
            
            # 설정 정보 출력 (존재하는 것만)
            cost_info = []
            if '광고매출' in df.columns:
                cost_info.append(f"광고매출={base_scenario.iloc[0]['광고매출']}원")
            if '7일연속보상' in df.columns:
                cost_info.append(f"7일연속보상={base_scenario.iloc[0]['7일연속보상']}원")
            if '추가보상' in df.columns:
                cost_info.append(f"추가보상={base_scenario.iloc[0]['추가보상']}원")
            
            if cost_info:
                print(f"설정: {', '.join(cost_info)}")
            
            # 확률 정보 출력
            prob_info = []
            for var in ['정답률', '광고시청률', '추가보상률', '재도전률']:
                if var in df.columns:
                    prob_info.append(base_scenario.iloc[0][var])
            
            if prob_info:
                print(f"확률: {', '.join(map(str, prob_info))}")
            
            if '주간순손익액' in df.columns:
                print(f"주간 순손익: {base_scenario.iloc[0]['주간순손익액']:,}원")
            if '수익성여부' in df.columns:
                print(f"수익성: {base_scenario.iloc[0]['수익성여부']}")
    
    # 비용/매출 변수별 수익 시나리오 분포
    if '수익성여부' in df.columns:
        profitable_df = df[df['수익성여부'] == '수익']
        
        if len(profitable_df) > 0:
            print(f"\n=== 비용/매출 변수별 수익 시나리오 분포 ===")
            
            for var in ['광고매출', '7일연속보상', '추가보상']:
                if var in df.columns:
                    var_dist = profitable_df[var].value_counts().sort_index()
                    print(f"{var}별 수익 시나리오 수:")
                    for value, count in var_dist.items():
                        print(f"  {value}: {count:,}개")
                    print()
    
    # CSV 저장
    filename = 'quiz_grid_search_results_extended.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n결과가 '{filename}'에 저장되었습니다.")
    
    return df

# 실행
if __name__ == "__main__":
    df = main()
    
    # DataFrame이 성공적으로 생성되었는지 확인
    if df is not None and len(df) > 0:
        print("\n" + "="*80)
        
        # 예시: 상위 10개 시나리오 추출
        try:
            best_analysis = extract_best_scenarios(df, n=10)
            
            if best_analysis:
                print("\n✅ 기본 시나리오 분석 완료")
            else:
                print("\n❌ 기본 시나리오 분석 실패")
        except Exception as e:
            print(f"\n❌ 기본 시나리오 분석 중 오류: {e}")
        
        # 예시: 특정 조건으로 시나리오 추출 (컬럼 존재 확인)
        print("\n" + "="*80)
        
        # 커스텀 기준 시나리오가 있는지 확인
        required_cols = ['광고매출', '7일연속보상', '추가보상', '정답률', '광고시청률', '추가보상률', '재도전률']
        if all(col in df.columns for col in required_cols):
            try:
                custom_base = {
                    '광고매출': 2,
                    '7일연속보상': 50,
                    '추가보상': 2,
                    '정답률': '60.0%',
                    '광고시청률': '60.0%',
                    '추가보상률': '80.0%',
                    '재도전률': '40.0%'
                }
                custom_analysis = extract_best_scenarios(df, base_scenario_params=custom_base, n=5, improvement_threshold=100000)
                
                if custom_analysis:
                    print("\n✅ 커스텀 시나리오 분석 완료")
                else:
                    print("\n❌ 커스텀 시나리오 분석 실패")
            except Exception as e:
                print(f"\n❌ 커스텀 시나리오 분석 중 오류: {e}")
        else:
            missing_cols = [col for col in required_cols if col not in df.columns]
            print(f"\n⚠️ 커스텀 분석을 위한 필수 컬럼 누락: {missing_cols}")
    else:
        print("\n❌ DataFrame 생성 실패")

=== 샘플 테스트 ===
테스트 결과 키: ['광고매출', '7일연속보상', '추가보상', '정답률', '광고시청률', '추가보상률', '재도전률', '일일_광고시청수', '일일_참여수', '일일_일반보상지급수', '일일_재도전성공수', '일일_추가보상수', '일일_7일연속보상수', '일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액', '광고매출액', '총매출액', '일일순손익액', '주간순손익액', '수익성여부']
광고매출 포함 여부: True
7일연속보상 포함 여부: True
추가보상 포함 여부: True
Grid Search 시작:
확률 변수: 7×7×5×7
비용/매출 변수: 5×10×3
총 시나리오: 257,250개
진행상황: 9,999개 완료 (3.9%)...
진행상황: 19,999개 완료 (7.8%)...
진행상황: 29,999개 완료 (11.7%)...
진행상황: 39,999개 완료 (15.5%)...
진행상황: 49,999개 완료 (19.4%)...
진행상황: 59,999개 완료 (23.3%)...
진행상황: 69,999개 완료 (27.2%)...
진행상황: 79,999개 완료 (31.1%)...
진행상황: 89,999개 완료 (35.0%)...
진행상황: 99,999개 완료 (38.9%)...
진행상황: 109,999개 완료 (42.8%)...
진행상황: 119,999개 완료 (46.6%)...
진행상황: 129,999개 완료 (50.5%)...
진행상황: 139,999개 완료 (54.4%)...
진행상황: 149,999개 완료 (58.3%)...
진행상황: 159,999개 완료 (62.2%)...
진행상황: 169,999개 완료 (66.1%)...
진행상황: 179,999개 완료 (70.0%)...
진행상황: 189,999개 완료 (73.9%)...
진행상황: 199,999개 완료 (77.7%)...
진행상황: 209,999개 완료 (81.6%)...
진행상황: 219,999개 완료 (85.5%)...
진행상황

In [3]:
df.columns

Index(['시나리오_ID', '광고매출', '7일연속보상', '추가보상', '정답률', '광고시청률', '추가보상률', '재도전률',
       '일일_광고시청수', '일일_참여수', '일일_일반보상지급수', '일일_재도전성공수', '일일_추가보상수',
       '일일_7일연속보상수', '일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액', '광고매출액',
       '총매출액', '일일순손익액', '주간순손익액', '수익성여부'],
      dtype='object')

In [7]:
df[(df['광고매출'] == 1) & (df['7일연속보상'] == 100) & (df['추가보상'] == 3)][['정답률','광고시청률', '추가보상률', '재도전률',
       '일일_광고시청수', '일일_참여수', '일일_일반보상지급수', '일일_재도전성공수', '일일_추가보상수',
       '일일_7일연속보상수', '일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액', '광고매출액',
       '총매출액', '일일순손익액', '주간순손익액', '수익성여부']]

,정답률,광고시청률,추가보상률,재도전률,일일_광고시청수,일일_참여수,일일_일반보상지급수,일일_재도전성공수,일일_추가보상수,일일_7일연속보상수,일반보상액,재도전보상액,추가보상액,7일연속보상액,총보상액,광고매출액,총매출액,일일순손익액,주간순손익액,수익성여부
49735,20.0%,20.0%,50.0%,20.0%,87728,100000,20000,3793,2379,17498,60000,3793,7137,4733750,4804680,87728,87728,-4716952,-4616164,손실
49736,20.0%,20.0%,50.0%,30.0%,92719,100000,20000,6228,2622,17498,60000,6228,7866,4733750,4807844,92719,92719,-4715125,-4603375,손실
49737,20.0%,20.0%,50.0%,40.0%,100870,100000,20000,9313,2931,17498,60000,9313,8793,4733750,4811856,100870,100870,-4710986,-4574402,손실
49738,20.0%,20.0%,50.0%,50.0%,113213,100000,20000,13278,3327,17498,60000,13278,9981,4733750,4817009,113213,113213,-4703796,-4524072,손실
49739,20.0%,20.0%,50.0%,60.0%,131853,100000,20000,18353,3835,17498,60000,18353,11505,4733750,4823608,131853,131853,-4691755,-4439785,손실
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51445,80.0%,80.0%,90.0%,40.0%,90253,100000,80000,6952,62606,17498,240000,6952,187818,4733750,5168520,90253,90253,-5078267,-7145369,손실
51446,80.0%,80.0%,90.0%,50.0%,92204,100000,80000,8879,63993,17498,240000,8879,191979,4733750,5174608,92204,92204,-5082404,-7174328,손실
51447,80.0%,80.0%,90.0%,60.0%,94345,100000,80000,10890,65440,17498,240000,10890,196320,4733750,5180960,94345,94345,-5086615,-7203805,손실
51448,80.0%,80.0%,90.0%,70.0%,96687,100000,80000,13018,66973,17498,240000,13018,200919,4733750,5187687,96687,96687,-5091000,-7234500,손실


In [19]:
df[
    (df['광고매출'] == 2) & 
    (df['7일연속보상'] == 10) & 
    (df['추가보상'] == 3)
    &(df['수익성여부'] == '수익')
][
    [
        '정답률', '광고시청률', '추가보상률', '재도전률',
        '일일_광고시청수', '일일_참여수', '일일_일반보상지급수', '일일_재도전성공수', '일일_추가보상수',
        '일일_7일연속보상수', '일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액', '광고매출액',
        '총매출액', '일일순손익액', '주간순손익액', '수익성여부'
    ]
].sort_values(by='주간순손익액', ascending=False)

,정답률,광고시청률,추가보상률,재도전률,일일_광고시청수,일일_참여수,일일_일반보상지급수,일일_재도전성공수,일일_추가보상수,일일_7일연속보상수,일반보상액,재도전보상액,추가보상액,7일연속보상액,총보상액,광고매출액,총매출액,일일순손익액,주간순손익액,수익성여부
55096,20.0%,80.0%,50.0%,80.0%,236582,100000,20000,35145,22058,17498,60000,35145,66174,473375,634694,473164,473164,-161530,1709540,수익
55061,20.0%,70.0%,50.0%,80.0%,231067,100000,20000,35145,19300,17498,60000,35145,57900,473375,626420,462134,462134,-164286,1690248,수익
55026,20.0%,60.0%,50.0%,80.0%,225553,100000,20000,35145,16543,17498,60000,35145,49629,473375,618149,451106,451106,-167043,1670949,수익
54991,20.0%,50.0%,50.0%,80.0%,220038,100000,20000,35145,13786,17498,60000,35145,41358,473375,609878,440076,440076,-169802,1651636,수익
54956,20.0%,40.0%,50.0%,80.0%,214524,100000,20000,35145,11029,17498,60000,35145,33087,473375,601607,429048,429048,-172559,1632337,수익
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55197,30.0%,40.0%,50.0%,40.0%,97459,100000,30000,11594,8318,17498,90000,11594,24954,473375,599923,194918,194918,-405005,5215,수익
55336,30.0%,80.0%,50.0%,30.0%,105851,100000,30000,7900,15160,17498,90000,7900,45480,473375,616755,211702,211702,-405053,4879,수익
55317,30.0%,70.0%,70.0%,50.0%,120995,100000,30000,16069,22573,17498,90000,16069,67719,473375,647163,241990,241990,-405173,4039,수익
55352,30.0%,80.0%,70.0%,50.0%,125602,100000,30000,16069,25798,17498,90000,16069,77394,473375,656838,251204,251204,-405634,812,수익


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257250 entries, 0 to 257249
Data columns (total 24 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   시나리오_ID     257250 non-null  object
 1   광고매출        257250 non-null  int64 
 2   7일연속보상      257250 non-null  int64 
 3   추가보상        257250 non-null  int64 
 4   정답률         257250 non-null  object
 5   광고시청률       257250 non-null  object
 6   추가보상률       257250 non-null  object
 7   재도전률        257250 non-null  object
 8   일일_광고시청수    257250 non-null  int64 
 9   일일_참여수      257250 non-null  int64 
 10  일일_일반보상지급수  257250 non-null  int64 
 11  일일_재도전성공수   257250 non-null  int64 
 12  일일_추가보상수    257250 non-null  int64 
 13  일일_7일연속보상수  257250 non-null  int64 
 14  일반보상액       257250 non-null  int64 
 15  재도전보상액      257250 non-null  int64 
 16  추가보상액       257250 non-null  int64 
 17  7일연속보상액     257250 non-null  int64 
 18  총보상액        257250 non-null  int64 
 19  광고매출액       257250 non-

In [ ]:
df 에서 

In [25]:
import numpy as np

# 모든 열을 숫자로 변환 (변환 불가한 값은 NaN으로 처리)
# df_numeric = 

df.to_excel('quiz_grid_search_results_extended.xlsx', index=False)